In [4]:
# ✅ 0. Kaggle 데이터 다운로드 설정
!pip install --upgrade kaggle --force-reinstall -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from google.colab import files
files.upload()  # kaggle.json 파일 직접 업로드

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"icdi050227","key":"6738515dc0795c218ea602dfa0ddc433"}'}

In [6]:
!mkdir -p ~/.kaggle
!mkdir -p /root/.config/kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json /root/.config/kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json

In [7]:
# ✅ Kaggle 데이터 다운로드
!kaggle competitions download -c state-farm-distracted-driver-detection -q


In [8]:
!unzip -oq /content/state-farm-distracted-driver-detection.zip -d /content/statefarm_data


In [ ]:
# ============================================================
# ✅ 1. 라이브러리 설치 및 GPU 설정
# ============================================================
!pip install timm -q

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.model_selection import KFold

In [ ]:
# ============================================================
# ✅ 2. GPU 확인
# ============================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU 이름: {torch.cuda.get_device_name(0)}')

사용 디바이스: cuda
GPU 이름: Tesla T4


In [ ]:
# ============================================================
# ✅ 3. 데이터 경로 설정 (Kaggle 다운로드 기준)
# ============================================================
base_dir = "/content/statefarm_data"
driver_csv_path = os.path.join(base_dir, "driver_imgs_list.csv")
train_dir = os.path.join(base_dir, "imgs", "train")
test_dir = os.path.join(base_dir, "imgs", "test")

print("CSV 존재?", os.path.exists(driver_csv_path))
print("Train 폴더 존재?", os.path.exists(train_dir))
print("Test 폴더 존재?", os.path.exists(test_dir))

CSV 존재? True
Train 폴더 존재? True
Test 폴더 존재? True


In [ ]:
# ============================================================
# ✅ 4. 하이퍼파라미터
# ============================================================
img_size = 224
batch_size = 64
num_classes = 10
num_epochs = 40
learning_rate = 0.001
num_workers = 2

In [ ]:

# ============================================================
# ✅ 5. 데이터 로드
# ============================================================
driver_df = pd.read_csv(driver_csv_path)
all_drivers = sorted(driver_df['subject'].unique())
n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [ ]:
# ============================================================
# ✅ 6. Dataset 정의
# ============================================================
class DriverDataset(Dataset):
    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images, self.labels = [], []

        if is_test:
            for img_name in os.listdir(data_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.images.append(os.path.join(data_dir, img_name))
        else:
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]
            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                img_path = os.path.join(data_dir, class_name, img_name)
                self.images.append(img_path)
                self.labels.append(int(class_name[1:]))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.is_test:
            return image, os.path.basename(self.images[idx])
        else:
            return image, self.labels[idx]


In [ ]:
# ============================================================
# ✅ 7. Transform 설정
# ============================================================
transform_train = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_eval = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# ============================================================
# ✅ 8. 학습 / 검증 함수
# ============================================================
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    return total_loss/total, 100*correct/total

def validate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return total_loss/total, 100*correct/total

In [ ]:
# --------------------------------------------------
# Fold 학습 함수 (✅ Fine-tuning + 자동 저장 기능 포함)
# --------------------------------------------------
def train_fold(fold_idx, train_drivers, val_drivers):
    print(f"\n========== Fold {fold_idx}/{n_folds} ==========")
    train_dataset = DriverDataset(train_dir, driver_df, train_drivers, transform_train)
    val_dataset = DriverDataset(train_dir, driver_df, val_drivers, transform_eval)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # ✅ torchvision AlexNet fine-tuning
    from torchvision import models
    model = models.alexnet(pretrained=True)

    # (1) 출력 레이어 교체
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

    # (2) 특징 추출부 동결
    for param in model.features.parameters():
        param.requires_grad = False

    # (3) 학습 가능한 파라미터만 optimizer에 전달
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=learning_rate
    )

    # (4) 손실함수 & device 설정
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)

    # ✅ (추가) 학습 기록 및 모델 저장 관련 초기화
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    model_path = f'alexnet_fold{fold_idx}_best.pth'

    # --------------------------------------------------
    # Epoch 루프
    # --------------------------------------------------
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f'Train Loss {train_loss:.4f}, Acc {train_acc:.2f}% | Val Loss {val_loss:.4f}, Acc {val_acc:.2f}%')

        # 🔹 학습 이력 기록
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # 🔹 5 epoch마다 임시 저장
        if (epoch + 1) % 5 == 0:
            temp_path = f'alexnet_fold{fold_idx}_epoch{epoch+1}.pth'
            torch.save(model.state_dict(), temp_path)
            print(f"🟢 {epoch+1} epoch까지 임시 저장 완료 → {temp_path}")

        # 🔹 최적 모델 저장 (Val Loss 기준)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f"✅ 새로운 최고 모델 갱신! {model_path} (Val Loss={val_loss:.4f})")

    # --------------------------------------------------
    # 학습 완료 후 결과 저장
    # --------------------------------------------------
    final_model_path = f'alexnet_fold{fold_idx}_final.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"\n💾 학습 완료! 최종 모델 저장: {final_model_path}")

    # 🔹 학습 이력 CSV 저장
    import pandas as pd
    history_df = pd.DataFrame(history)
    history_csv = f'alexnet_fold{fold_idx}_history.csv'
    history_df.to_csv(history_csv, index=False)
    print(f"📈 학습 이력 저장 완료: {history_csv}")

    return model_path, history


In [ ]:
# ============================================================
# ✅ 10. Fold 2 학습 예시 (자동 저장 기능 포함)
# ============================================================

# Fold 정보 설정
fold_info = list(kfold.split(all_drivers))[1]
train_drivers = [all_drivers[i] for i in fold_info[0]]
val_drivers = [all_drivers[i] for i in fold_info[1]]

# 모델 학습 및 저장 함수 실행
model_path, history = train_fold(2, train_drivers, val_drivers)








========== Fold 2/5 ==========

Epoch 1/40


Train Loss 0.7232, Acc 75.05% | Val Loss 1.2292, Acc 67.12%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=1.2292)

Epoch 2/40


Train Loss 0.2801, Acc 90.82% | Val Loss 1.1968, Acc 68.22%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=1.1968)

Epoch 3/40


Train Loss 0.2416, Acc 92.40% | Val Loss 1.1125, Acc 68.57%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=1.1125)

Epoch 4/40


Train Loss 0.2093, Acc 93.62% | Val Loss 1.7595, Acc 60.01%

Epoch 5/40


...

Train Loss 0.0773, Acc 98.15% | Val Loss 1.5456, Acc 67.00%

Epoch 37/40


Train Loss 0.0892, Acc 97.89% | Val Loss 1.2146, Acc 71.87%

Epoch 38/40


Train Loss 0.0865, Acc 97.95% | Val Loss 0.9363, Acc 76.59%
✅ 새로운 최고 모델 갱신! alexnet_fold2_best.pth (Val Loss=0.9363)

Epoch 39/40


Train Loss 0.0778, Acc 98.08% | Val Loss 1.2228, Acc 72.62%

Epoch 40/40


Train Loss 0.0833, Acc 97.99% | Val Loss 1.5284, Acc 68.12%
🟢 40 epoch까지 임시 저장 완료 → alexnet_fold2_epoch40.pth

💾 학습 완료! 최종 모델 저장: alexnet_fold2_final.pth
📈 학습 이력 저장 완료: alexnet_fold2_history.csv


In [ ]:
# ============================================================
# ✅ 11. 테스트 예측 + 제출 CSV 생성
# ============================================================
print("\n📢 테스트 데이터 예측 시작...")

test_dataset = DriverDataset(test_dir, driver_df, [], transform_eval, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# ✅ timm 대신 torchvision으로 변경
from torchvision import models

model = models.alexnet(pretrained=False)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# ✅ 학습된 파라미터 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

predictions, filenames = [], []
with torch.no_grad():
    for images, names in tqdm(test_loader, desc="Predicting"):
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        predictions.append(probs.cpu().numpy())
        filenames.extend(names)

preds = np.vstack(predictions)
class_cols = [f'c{i}' for i in range(num_classes)]
submission = pd.DataFrame(preds, columns=class_cols)
submission.insert(0, 'img', filenames)

submission_file = '/content/alexnet_fold2_submission.csv'
submission.to_csv(submission_file, index=False)
print(f"\n✅ 제출 파일 생성 완료: {submission_file}")
print(submission.head())



📢 테스트 데이터 예측 시작...


Predicting: 100%|██████████| 1246/1246 [06:30<00:00,  3.19it/s]



✅ 제출 파일 생성 완료: /content/alexnet_fold2_submission.csv
             img            c0            c1            c2            c3  \
0  img_16561.jpg  2.537672e-03  2.256016e-02  1.699504e-03  4.996450e-05   
1  img_77002.jpg  1.628776e-03  1.781334e-05  7.400337e-05  9.857650e-01   
2  img_88996.jpg  6.426714e-02  6.894591e-01  9.172354e-02  1.969100e-02   
3  img_15767.jpg  5.953398e-02  1.280020e-01  1.932308e-02  1.573110e-02   
4  img_53781.jpg  7.088267e-26  1.631518e-40  1.298115e-31  2.203908e-33   

             c4        c5            c6            c7            c8  \
0  7.280873e-05  0.000455  1.858340e-03  6.374775e-01  4.482256e-02   
1  4.458329e-03  0.007002  2.063563e-06  4.166032e-04  1.693458e-04   
2  4.567990e-03  0.005871  1.872620e-02  2.959376e-02  3.614706e-02   
3  1.148526e-02  0.010449  2.306456e-02  2.384050e-02  6.107399e-01   
4  8.775504e-27  1.000000  1.608425e-27  7.172655e-30  1.107024e-18   

             c9  
0  2.884669e-01  
1  4.659258e-04  
2  3.995